# 3. Thermodynamic variability analysis (TVA)

There exists multiple flux states that define the same optimum. TVA predicts the ranges of variables such as metabolic fluxes, Gibbs energy of reactions and metabolite concentrations.

As described previously, you can solve the TVA problem on either the MILP or MIQCP. Although, MILP is faster compared to solving MIQCP, MIQCP is thermodynamically consistent as they drawn from multivariate distribution. 

There are three different functions for running TVA in multiTFA. One for the MILP, the remaining two for MIQCP, depending on the solver you use (either Gurobi or Cplex).

By default, TVA is performed on all the variables in a model unless `variable_list` is specified. Please note, `variable_list` is a `list` of `str`. For example reactons id's or names of Gibbs energy variables etc.


In [ ]:
from multiTFA import analysis
rxn_vars = [rxn.id for rxn in t_model.reactions if rxn.id not in t_model.Exclude_reactions] # Flux variable names
dg_vars = [rxn.delG_forward.name for rxn in t_model.reactions if rxn.id not in t_model.Exclude_reactions] # Gibbs energy variable names

ranges = analysis.variability(t_model, variable_list=rxn_vars + dg_vars) # TVA using MILP

ranges_miqc = analysis.variability_legacy_cplex(t_model,variable_list=vars_analysis) # TVA using MIQCP with cplex

ranges_miqc = analysis.variability_legacy_gurobi(t_model,variable_list=vars_analysis) # TVA using MIQCP with Gurobi

Setting the parameter `fraction_of_optim=0.9` would give variable ranges at 90% optimality of specified variable optimality. The variable can be specified by Setting `biomass_rxn` parameter.  

In [ ]:
analysis.variability_legacy_cplex(t_model,variable_list=vars_analysis, biomass_rxn='BIOMASS_Ec_iJO1366_core_53p95M', fraction_of_optim=0.9)

Please note, runnig TVA on genome scale model is computationally expensive, especially using MIQC problems. We have observed that relaxing the `Gap` parameter helped achieving faster run times. 